# Creation of AWS IoT Sitewise Assets at scale

### ⛳ Objective
To work with a large number of assets or asset models, use bulk operations to bulk import and export resources to a different location. In this Notebook we will do the following
1. Execute **Metedata Bulk Import:** You have a data file that defines assets and asset models and upload it in an Amazon S3 bucket, and use bulk import to create or update them in AWS IoT SiteWise. 
2. **Shopfloor Connectivity Framwork** Deployment : You will deploy Shopfloor Connectivity Framework (SFC) and also configure it to send data to the asstes you have created in the previous step.
3. **GenerativeAI** Playground : You will execute the **Metedata Bulk Import** agian but this time you will use the artefacts and configuration files created in the previous Notebook.
4. **Visualisation:** You will visualise the assets you have created using AWS IoT Sitewise Monitor. You will learn how to create the Portal programatically and then create dashboards.

### 🗝️ Key concept refreshers

Before we dive into the practical application, let's quickly review the essential concepts:

1. [**AWS IoT SiteWise Bulk Import :**](https://docs.aws.amazon.com/iot-sitewise/latest/userguide/bulk-operations-assets-and-models.html) AWS IoT SiteWise Bulk Import enables the large-scale ingestion of historical and real-time equipment data into SiteWise for analysis and monitoring. It simplifies the import process by supporting structured data ingestion in bulk, especially useful for legacy systems. Learn more about bulk import.
2. **Shopfloor Connectivity :** Shopfloor Connectivity connects factory-floor equipment (e.g., machines, sensors) to digital systems for real-time data collection and monitoring. This integration, using protocols like OPC-UA, Siemens S7 and Modbus, improves operational visibility and supports predictive maintenance.
3. [**AWS IoT SiteWise Monitor :**](https://docs.aws.amazon.com/iot-sitewise/latest/userguide/bulk-operations-assets-and-models.html) AWS IoT SiteWise Monitor is a tool that allows users to create and share real-time dashboards for monitoring industrial data. It provides a no-code interface for easy dashboard customization and secure access for team members.

<div class="alert alert-block alert-info">
<b>💡</b> You perform bulk operations in AWS IoT SiteWise by calling operations in the AWS IoT TwinMaker API. You can do this without setting up AWS IoT TwinMaker or creating an AWS IoT TwinMaker workspace. All you need is an Amazon S3 bucket where you can place your AWS IoT SiteWise content.
</div>



### &#x1F4DA; Libraries

In [5]:
%%capture config_logs
!py -m pip install pickleshare

import boto3
import json
import time
import os
from datetime import datetime

from ec2_metadata import ec2_metadata
from sitewise_helpers import *
import time

### &#x2699; Configuration for AWS IoT Sitewise


In [ ]:

aws_account = boto3.client('sts').get_caller_identity()['Account']
aws_region = ec2_metadata.region
aws_stack_name = 'uns-scale-connectivity-workshop'
twinmakerclient = boto3.client('iottwinmaker', region_name=aws_region)
s3client = boto3.client('s3')
bucketname = f'reinvent-{aws_stack_name}-{aws_account}-{aws_region}'  
roleArn = f'arn:aws:iam::{aws_account}:role/data_import_role_workshop-{aws_stack_name}'
print(f'aws_region: {aws_region}')
print(f'aws_account: {aws_account}')
print (f'aws_bucketname: {bucketname}')

## 1. AWS IoT Sitewise Bulk Operations to create assets and models at scale


<div class="alert alert-block alert-info">
<span style="font-size: 1.5em;">💡</span> In the last <em>Notebook 02</em>, we generated three configuration files. Generally, any Large language Model (LLM) response needs to be tested and quality checked against a desired outcome before deployment in Production. So before we start working with the files generated by GenAI, let's first try to understand how bulk import works with a pre-configured schema.</div>

In this step, we will use <span style="font-family: monospace;">complete_assets_models_definitions.json</span> where the following things are configured into the AWS IoT SiteWise schema
1. Asset Models
2. Asset Models Properties and Metrics
3. Assets
4. Hierarchy Definition

### 1.1 Create Bulk Import Job for AWS IoT Sitewise

To run a bulk import job is a three step process :
1. Preparing the schema file to import
2. Upload the prepared file to Amazon S3
3. Execute Bulk import Job

In the next cell, we create a function definition to execute bulk import that takes the schema definition file as an argument

In [ ]:
# Fucnction definition to execute a meta data bulk import
def meta_data_import(filename):
    os.chdir('C://Users//Administrator//Documents//jupyter-notebook')
    prefix = 'data/'

    # 1. Upload the config files to S3
    s3client.upload_file(filename, bucketname, prefix + filename)
    print("Uploaded " + filename + " to " + bucketname + " with prefix " + prefix)

    job_id = f'Bulk_Import_ID_{int(datetime.now().timestamp())}'
    s3_location = f'arn:aws:s3:::{bucketname}/{prefix}{filename}'
    
    # 2. Execute Bulk Import API Call to AWS IoT Sitewise
    print(job_id)
    twinmakerresponse = twinmakerclient.create_metadata_transfer_job(
        metadataTransferJobId = job_id,
        sources=[{
            'type': 's3',
            's3Configuration': {
            'location': s3_location
                }
            }],
        destination={
                'type': 'iotsitewise'
            }
        )
    return job_id

Now we will call this function, where we will pass a preconfigured schema with AWS IoT Sitewise asset models, assets and assets hierarchy 

In [ ]:
# Function call to execute meta data bulk import
sitewise_schema = "complete_asset_models_definition.json"
import_job_id = meta_data_import(sitewise_schema)

<div>
<span style="font-size: 1.5em;">📋</span> As you wait for the next execution to complete, go through the <span style="font-family: monospace;">complete_asset_models_definition.json</span> template and the response files generated by Amazon Bedrock <em>Notebook 02</em> <ul>
 <li>What difference do you see ?</li>
 <li>How similar are they to each other?</li>
 <li>Are you confident to excute another Bulk import job using the LLM's response ?</li></ul> 
</div>

### 1.1 Check Bulk Import Job Progress

Before moving ahead we need to wait until the bulk import job is completed. You can check this either on the AWS Console or you can use the status response method offered tou you by AWS IoT Sitewise to check the programatically. In both cases, please wait until the execution is complete!

To check the status from this Notebook, please execute the next two cells where you create the function defination and perform function call to check status

In [9]:
# Function definition to check the progress of the bulk import job
def check_bulk_import_progress(job_id):
    while True:
        res = twinmakerclient.get_metadata_transfer_job(metadataTransferJobId=job_id)
        state = res["status"]["state"]
        if state in ('RUNNING', 'COMPLETED'):
            try:
                progress = res["progress"]
                print(f'Status: {state} | Total: {progress["totalCount"]}, Suceeded: {progress["succeededCount"]}, Skipped: {progress["skippedCount"]}, Failed: {progress["failedCount"]}')                
            except:
                print(f'Status: {state}')
        elif state == 'ERROR':
            print(f'Status: {state} | Report URL: {res["reportUrl"]}')
            break
        else:
            print(f'Status: {state}')
        if state == 'COMPLETED': break
        time.sleep(20) # Check status every 20 seconds

In [ ]:
# Function call to check status of metadata bulk import job
check_bulk_import_progress(import_job_id)

<div class="alert alert-block alert-info">
<span style="font-size: 1.5em;">💡</span> You can also look in the AWS IoT SiteWise Console as the assets are being created live!
</div>


### 2. Deploy Shopfloor Connectivity framework to send data to AWS IoT SiteWise

Let's now revisit SFC that got introduced in the previous chapter.

SFC requires two main types of configurations:
1. Source configurations: Define how to collect data from industrial equipment (e.g., protocols like SIEMENS S7, Ethernet/IP, MQTT)
2. Target configurations: Specify how to send data to AWS services (e.g., AWS IoT SiteWise, Amazon Timestream, Amazon S3)

[It can be deployed in two modes](https://github.com/aws-samples/shopfloor-connectivity/blob/mainline/docs/README.md#in-process-and-ipc-deployment-models): **in-Process** and **Inter-Process-Communication (IPC)**.
In this workshop we are using the **In-Process** mode as it within a single, unified process.

#### 2.1 Create SFC configuration based on TIA export files

Two Python scripts are used to create a valid SFC source and target config package:

1. `scripts/create_sfc_config_files.py`
2. `scripts/sfc/s7tia2sfc.py`

These scripts use methods to extract device and data block information from the TIA exports. The extracted data is then converted into a JSON dictionary format that can be used by the SFC framework.
The resulting SFC config package is stored in the `imports/sfc` folder.

The next step creates JSON config files and executables for running SFC locally. These files are then moved to the ```C:/``` drive.

In [ ]:
import os
import shutil
os.chdir('C://Users//Administrator//Documents//jupyter-notebook//scripts')
!py create_sfc_config_files.py --region {aws_region}
shutil.copytree(r"C:\Users\Administrator\Documents\jupyter-notebook\imports\sfc\CarFactory", r"C:\CarFactory", dirs_exist_ok=True)

#### 2.2 Install and execute the SFC

Next, we will execute the SFC installer. This will downloads protocol adapters and target adapters from the SFC GitHub releases. These are necessary components for the SFC execution.

In [ ]:
!powershell.exe -Command "Set-Location -Path C:\CarFactory ; ./sfc-standalone-install.bat"

We'll now run the SFC. This process will ingest all source data into [disassociated Sitewise streams](https://docs.aws.amazon.com/iot-sitewise/latest/userguide/manage-data-streams-method.html). 
As the SFC executes, watch for indicators of successful data transmission. You should see logs or messages confirming that data is being sent to AWS IoT SiteWise. Disassociated streams allow for rapid data ingestion without requiring a predefined asset structure. 
This approach enables us to collect data quickly and explore it before finalizing our asset models.

**Important Note:** The following code will initiate a PowerShell window to run the SFC component. This window must remain open for continuous data ingestion. Monitor the PowerShell window for real-time ingestion status and any potential error messages.

In [43]:
os.chdir('C://CarFactory')
!powershell.exe -Command "Set-Location -Path C:\CarFactory ; Start-process powershell.exe -ArgumentList 'C:\CarFactory\sfc-standalone-runner.bat'"

Execute the cell below to implement a verified AWS IoT SiteWise target configuration for SFC. This will ingest the near real-time data to the created assets.

In [ ]:
import shutil
shutil.copyfile(r'C:\Users\Administrator\Documents\jupyter-notebook\include_generated_swtarget.json', r"C:\CarFactory\include_generated_swtarget.json")
# IMPORTANT: check region attribute in C:\CarFactory\include_generated_swtarget.json - defaults to us-west-2

Follow the workshop's guided steps to navigate to the AWS IoT SiteWise console and monitor the live data ingestion process under the asset's ```Latest Value``` area. 

<div>
<span style="font-size: 1.5em;">&#x1F3C6;</span><b>Congratulations!</b>
You now have a Unified Namespace setup in the Cloud. Now let's visualise it using AWS IoT SiteWise Monitor.
</div>

### 3. 📈 Visualisation of data in Unified Namespace using AWS IoT Sitewise Monitor

AWS IoT SiteWise Monitor is a feature of AWS IoT SiteWise that provides portals in the form of managed web applications. You can use these applications to view and share your operational data and alarms. You can see data from your processes, devices, and equipment that are connected to AWS IoT SiteWise. Domain experts, such as process engineers, can use these portals to quickly get insights into their operational data to understand device and equipment behavior. They can use these insights to improve efficiency of devices, processes, or equipment and innovate on new initiatives. Operators can monitor data with alarms and respond to alarms when devices and equipment perform sub-optimally.

We will now create AWS Sitewise Monitor Portal to visualise data

#### 3.1 Create AWS IoT Sitewise Monitor

In [ ]:
portal_Name = 'reInvent Car Factory'                                    # Can be changed per liking
email_id = 'abcde@gmail.com'                                            # Provide your personal email id

if check_if_portal_exists(portal_Name):
    print(f"A portal with the name '{portal_Name}' already exists.")
    
else:
    portal_meta_data = create_visualisation(portal_Name,email_id)
    %store portal_meta_data

<div class="alert alert-block alert-info">
<span style="font-size: 1.5em;">📋</span> Now follow the instruction in the workshop to create dashboards and visualise KPIs of the factory that you have created.
</div>


### 4. 	🦾 GenerativeAI Playground (Optional)

Now you should have an understanding of how AWS IoT SiteWise metadata bulk imports work, now let's try to execute the same steps as before with acutal configration files and schema that were generated by Amazon Bedrock in the previous Notebook. In the first step you had created the complete hierarchy, we will reuse Asset Models, Properties and Metrics

<div class="alert alert-block alert-info padding-left: 2em;">
<span style="font-size: 1.5em;">⏱️</span>  Each metadata bulk operation execution takes 2 mins on average. So, please be patient and make sure to check the AWS Console after every execution to see the results. Please refresh your browser if needed
</div>

#### 4.1 Initialise your playground
Before we start lets delete assets and the hierarchy definition from the previous steps, so that we can create it new in the next steps

In [11]:
from sitewise_helpers import *
delete_child_assets(classic=True)

<div style="padding-left: 2em;">&#x2611; All Cell Units or child assets would be deleted</div>

#### 4.2 Generate GenAI AWS IoT SiteWise assets
Next, using the asset schema JSON <span style="font-family: monospace;"> genai_sitewise_assets_schema.json</span> created by Amazon Bedrock in the last Notebook, execute the metadata bulk import

In [ ]:
sitewise_schema = "genai_sitewise_assets_schema.json"
import_job_id = meta_data_import(sitewise_schema)

check_bulk_import_progress(import_job_id)

<div style="padding-left: 2em;">&#x2611; New GenAI Cell Units or child assets are created</div>

#### 4.3 Generate GenAI AWS IoT SiteWise assets hieararchy

You will see new assets are created with <span style="font-family: monospace;">GenAI</span> prefix as expected. Next, execute the next cell to initiate another bulk import to define hierarchies for the assets that you have created and this time with the config file <span style="font-family: monospace;">genai_sitewise_hierarchy_schema.json</span>

In [ ]:
sitewise_schema = "genai_sitewise_hierarchy_schema.json"
import_job_id = meta_data_import(sitewise_schema)

check_bulk_import_progress(import_job_id)

<div style="padding-left: 2em;">&#x2611; New GenAI asset hierarchies are created</div>

#### 4.4 Configure SFC for GenAI assets
Now we will **manually** override the previous AWS IoT SiteWise config for SFC with the configuration that was generated by Amazon Bedrock. For this we will use the config file <span style="font-family: monospace;"> genai_sfc_sitewise_target_conf.json</span>. SFC will update config automatically during runtime.

You should see live data on AWS IoT SiteWise Console and Monitor. Also observe the logs in the powershell

In [ ]:
import shutil
shutil.copyfile(r'C:\Users\Administrator\Documents\jupyter-notebook\genai_sfc_sitewise_target_conf.json', r"C:\CarFactory\include_generated_swtarget.json")

<div style="padding-left: 2em;">&#x2611; SFC configured and data is streamed to AWS IoT Sitewise</div>

<div>
<p align="center">
        <span style="font-size: 1.5em;">🏁</span><em> You have now completed this Workshop! Please don't forget to cleanup the resources. Check the worshop documentation as well <em><span style="font-size: 1.5em;">🏁</span>
    </p></div>

## 🧹Cleanup

<div class="alert alert-block alert-danger">
<span style="font-size: 1.5em;">&#x2757;</span> Excecute the Clean-up cells sequentially only at the end of the workshop. if you execute it any earlier you will have to start over from the beginning of this notebook
</div>

1. Deletes all Sitewise resources you have created

In [ ]:
from sitewise_helpers import *
delete_all_sitewise_elements()

2. Deletes the AWS IoT Sitewise monitor you created earlier

In [ ]:
from sitewise_helpers import *
%store -r portal_meta_data
delete_visualisation(portal_meta_data)

3. Deletes all contents in the S3 bucket

In [ ]:
delete_all_objects(bucketname)